# Qdrant Reader/Writer Demo

Quickstart for moving vectors and payloads between Spark and a Qdrant collection using the spark-fuse connector.

> Prerequisites: a running Qdrant instance (e.g., `docker run -p 6333:6333 qdrant/qdrant`) and network access from the Spark driver to it.

In [ ]:
import json
from pyspark.sql import Row

from spark_fuse.spark import create_session
from spark_fuse.io import (
    QDRANT_CONFIG_OPTION,
    QDRANT_FORMAT,
    build_qdrant_config,
    build_qdrant_write_config,
    register_qdrant_data_source,
)

spark = create_session(app_name="spark-fuse-qdrant-demo")
register_qdrant_data_source(spark)

endpoint = "http://localhost:6333"  # adjust to your Qdrant host
collection = "demo_vectors"

# Example dataset to write
vectors = spark.createDataFrame(
    [
        Row(id=1, embedding=[0.1, 0.2, 0.3], name="alpha"),
        Row(id=2, embedding=[0.2, 0.3, 0.4], name="beta"),
    ]
)

# Write points to Qdrant
write_cfg = build_qdrant_write_config(
    endpoint,
    collection=collection,
    id_field="id",
    vector_field="embedding",
    payload_fields=["name"],
)
(
    vectors.write.format(QDRANT_FORMAT)
    .option(QDRANT_CONFIG_OPTION, json.dumps(write_cfg))
    .mode("append")
    .save()
)

# Read points back (include vectors)
read_cfg = build_qdrant_config(
    spark,
    endpoint,
    collection=collection,
    source_config={"with_vectors": True},
)
points = (
    spark.read.format(QDRANT_FORMAT)
    .option(QDRANT_CONFIG_OPTION, json.dumps(read_cfg))
    .load()
)
points.show(truncate=False)


Stop Spark when you are done.

In [ ]:
spark.stop()
